In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
print("""
This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      """)


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [4]:
%matplotlib inline
fig_width_pt = 525  # Get this from LaTeX using \showthe\columnwidth
fig_width_pt = 618  # Get this from LaTeX using \showthe\columnwidth
fig_width_pt = 1024  # Get this from LaTeX using \showthe\columnwidth
ppi = 72.27 # (constant) definition of the ppi = points per inch
inches_per_pt = 1.0/ppi  # Convert pt to inches
#inches_per_cm = 1./2.54
figwidth = fig_width_pt*inches_per_pt  # width in inches
phi = (np.sqrt(5) + 1. ) /2 # golden ratio is good for your eyes
dpi_export = 600

In [5]:
import datetime
print(datetime.datetime.now().date().isoformat())

2019-03-14


In [6]:
filename = '../data/' + datetime.datetime.now().date().isoformat()

In [7]:
import torch

In [8]:
import easydict
args = easydict.EasyDict(
                        # MNIST
                        w=28,
                        minibatch_size = 100,  # quantity of examples that'll be processed
                        train_batch_size=60000, # train
                        test_batch_size=1000, 
                        noise_batch_size=1000, 
                        mean=0.1307, 
                        std=0.3081, 
                        # display
                        N_pic = 128,
                        offset_std = 30, #
                        offset_max = 35, #
                        noise=1., #0 #
                        contrast=0.8, #1 #
                        sf_0=0.2,
                        B_sf=0.3,
                        # foveation
                        N_theta = 6,
                        N_azimuth = 16,
                        N_eccentricity = 10,
                        N_phase = 2,
                        N_X = 128,
                        N_Y = 128,
                        rho = 1.41,
                        # network
                        bias_deconv=True,
                        p_dropout=.5,
                        dim1=1000,
                        dim2=1000,
                        # training
                        loss_func = torch.nn.BCEWithLogitsLoss(), #torch.nn.CrossEntropyLoss()
                        lr = 1e-4, #1e-3  #0.05
                        do_adam=True,
                        epochs=40,
                        bn1_bn_momentum=0.5,
                        bn2_bn_momentum=0.5,
                        momentum=0.1,    
                        n_epochs=10,
                        # simulation
                        num_processes=1,
                        no_cuda=True,
                        log_interval=100, # period with which we report results for the loss
                        verbose = 1,
                        filename=filename,
                        seed=2019,
                        )

In [9]:
if True:
    args.train_batch_size = 100
    args.lr = 1e-2
    #args.noise = .5
    #args.contrast = .9
    #args.p_dropout = 0.
    args.epochs = 80
    #args.test_batch_size = 20
    #args.offset_std = 8

In [10]:
print(args)

{'w': 28, 'minibatch_size': 100, 'train_batch_size': 100, 'test_batch_size': 1000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 35, 'noise': 1.0, 'contrast': 0.8, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'N_X': 128, 'N_Y': 128, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.5, 'dim1': 1000, 'dim2': 1000, 'loss_func': BCEWithLogitsLoss(), 'lr': 0.01, 'do_adam': True, 'epochs': 80, 'bn1_bn_momentum': 0.5, 'bn2_bn_momentum': 0.5, 'momentum': 0.1, 'n_epochs': 10, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-14', 'seed': 2019}


In [11]:
from retina import Display
d = Display(args)

In [12]:
d.loader_train.batch_size

100

In [13]:
from what import WhatNet
model_path = "../data/MNIST_cnn.pt"
What_model = torch.load(model_path)

In [16]:
%%writefile train.py
import torch
import easydict

args = easydict.EasyDict({'w': 28, 'minibatch_size': 100, 'train_batch_size': 100, 'test_batch_size': 1000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 35, 'noise': 1.0, 'contrast': 0.8, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'N_X': 128, 'N_Y': 128, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.5, 'dim1': 1000, 'dim2': 1000, 'loss_func': torch.nn.BCEWithLogitsLoss(), 'lr': 0.01, 'do_adam': True, 'epochs': 80, 'bn1_bn_momentum': 0.5, 'bn2_bn_momentum': 0.5, 'momentum': 0.1, 'n_epochs': 10, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-14_b', 'seed': 2019})

from retina import Display, Retina
from where import Where, WhereNet
from what import WhatNet
where = Where(args, Display(args), Retina(args))
filename_train = args.filename + '_train.pt'
where.train(filename_train)


Overwriting train.py


## Version used

In [14]:
%load_ext watermark

In [15]:
%watermark -i -h -m -v -p numpy,matplotlib,torch  -r -g -b

2019-03-14T15:33:32+01:00

CPython 3.7.2
IPython 7.3.0

numpy 1.16.2
matplotlib 3.0.2
torch 1.0.1.post2

compiler   : Clang 10.0.0 (clang-1000.11.45.5)
system     : Darwin
release    : 18.2.0
machine    : x86_64
processor  : i386
CPU cores  : 36
interpreter: 64bit
host name  : fortytwo
Git hash   : d0db91898d99ee32fca27c07f35c4656d904eb17
Git repo   : https://github.com/laurentperrinet/WhereIsMyMNIST
Git branch : master
